In [1]:
#Intro to Web Scraping and BeautifulSoup
#https://www.youtube.com/watch?v=XQgXKtPSzUI

#exporting to excel
#df.to_excel("check.xlsx")

import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
#import re #for splitting strings using multiple delimiters
import time
import random

names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

In [2]:
#When populating species_XXX variables, make sure that they are all lowercase; to make comparison easier
#Common name on left side, taxonomy on right
species_vft = ['venus','flytrap','flytraps','dionaea','muscipula']
species_neps = ['tropical','asian','nepenthes']
species_sarracenia = ['american','sarracenia']
species_sundew = ['sundew','sundews','drosera']
species_utric = ['bladderwort','bladderworts','utricularia']
species_pings = ['butterwort','butterworts','pinguicula']
species_cephs = ['australian','cephalotus']
species_darlingtonia = ['cobra','lily','lilies','darlingtonia']
species_heli = ['sun','heliamphora']
#species_special = ['specimen plants','carnivero exclusives']

species_all = species_vft + species_neps + species_sarracenia + species_sundew + species_utric \
            + species_pings + species_cephs + species_darlingtonia + species_heli
    
#function to check type of species
#name is the name of the item as a string, returns a string of the type of species
def check_species(name):
    #cleaning: replacing special characters with spaces, all lowercase then splitting words into list
    kind_list = name.strip().replace('(','').replace(')','').replace('[','').replace(']','').replace(',','')
    kind_list = kind_list.lower().split(' ')
    
    n = len(kind_list)
    if n == 0:
        print('error: length of kind is 0, cannot loop')
        return 'error'
    
    #will try each element in the kind list and see if it is in species_XXX
    #if not, will keep looping over the length of n
    for i in range(n):
        if kind_list[i] in species_vft:
            return 'dionaea muscipula'
        elif kind_list[i] in species_neps:
            return 'nepenthes'
        elif kind_list[i] in species_sarracenia:
            return 'sarracenia'    
        elif kind_list[i] in species_sundew:
            return 'drosera'
        elif kind_list[i] in species_pings:
            return 'pinguicula'
        elif kind_list[i] in species_utric:
            return 'utricularia'
        elif kind_list[i] in species_cephs:
            return 'cephalotus'
        elif kind_list[i] in species_darlingtonia:
            return 'darlingtonia'
        elif kind_list[i] in species_heli:
            return 'heliamphora'
        elif i == (n-1):
            #when list of strings are not found in species_XXX, return other
            return 'other'

In [53]:
[len(names),len(prices),len(saleprices),len(soldouts),len(stores),len(species)]

[1, 1, 1, 0, 1, 1]

In [70]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Pearl River Exotics
URL = 'https://www.pearlriverexotics.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('a',attrs={'class':'site-nav__link site-nav__link--main'})
links = links[:len(links)-1] #excluding last link "carnivorous plants for beginners'

for link in links:
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path

    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    #finds how many pages to loop over
    try:
        page_count = soup.find('li',attrs={'class':'pagination__text'}).text.strip()
        page_count = page_count[len(page_count)-1:] #grabs how many pages to loop through
        page_count = int(page_count)
    except (AttributeError,IndexError):
        page_count = 1 #only 1 page

    for page_number in range(page_count):
        #wait random time between 0-5 seconds before scraping data
        r = random.randint(0,5)
        time.sleep(r)
    
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('a',attrs={'class':'grid-view-item__link'})

        for container in containers:
            name = container.find('div',attrs={'class':'h4 grid-view-item__title'}).text.strip()
            specie = check_species(name)

            price = container.find('span',attrs={'class':'product-price__price'}).text.strip()
            price = price[1:] #remove the 1st character of the string '$'
    
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Pearl River Exotics']
            saleprices = saleprices + [''] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!change when on sale
            species = species + [specie]
            
            #looks for sold out items
            try:
                soldout = container.find('span',attrs={'class':'product-price__sold-out'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
        
        #once page is scraped, grabs the URL for next page
        try:
            path = soup.findAll('a',attrs={'class':'btn btn--secondary btn--narrow'})
            path = path[len(path)-1] #gets the link to the next page
            path = path['href']
            URL2 = URL + path
        except (AttributeError,IndexError):
            continue #nothing

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df = pd.DataFrame(d)
df

,Name,Price,Sale Price,Sold Out,Species,Store
0,72+ Hour Heat Pack - For shipping with plants,4.00,,,other,Pearl River Exotics
1,Cephalotus follicularis,25.00,,sold out,cephalotus,Pearl River Exotics
2,Nepenthes burbidgeae BE-3834 (SEED-GROWN),59.00,,,nepenthes,Pearl River Exotics
3,Nepenthes burbidgeae x aristolochioides BE-3784,29.00,,,nepenthes,Pearl River Exotics
4,Nepenthes burkei x veitchii BE-3934,16.00,,,nepenthes,Pearl River Exotics
5,Nepenthes eymae BE-3736,30.00,,,nepenthes,Pearl River Exotics
6,Nepenthes glabrata x hamata - BE-3912,45.00,,,nepenthes,Pearl River Exotics
7,Nepenthes glandulifera BE-3766 *Seed-grown*,30.00,,,nepenthes,Pearl River Exotics
8,Nepenthes glandulifera BE-3879 (Single best cl...,30.00,,,nepenthes,Pearl River Exotics
9,Nepenthes lingulata BE-3463,59.00,,,nepenthes,Pearl River Exotics


In [71]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#California Carnivores
URL = 'https://www.californiacarnivores.com/'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
soup = soup.find('ul',attrs={'class':'sidebar-module__list'})
links = soup.findAll('a')
links = [links[i] for i in range(len(links)) if i not in (0,1,3,13,16,22,26,30,31,32,33,34,35,36,37)]
#exluding following pages: plant collections (0), easy to grow (1), drosera (3), sarracenia (13),
#nepenthes (16), pinguicula (22), utricularia (26), waterwheel, ..., gifts (30-37)

for link in links:
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page_next = True
    while page_next:
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half'})
        containers_sale = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half on-sale'})

        #grabs information of normal items
        for container in containers:
            name = container.find('p',attrs={'class':'grid-link__title'}).text.strip()
            specie = check_species(name)

            price = container.find('p',attrs={'class':'grid-link__meta'}).text.strip()
            separate_index = price.find('$') #look for 1st instance of $
            price = price[separate_index+2:] #remove the all characters before and including the string '$ '
    
            names = names + [name]
            prices = prices + [price]
            saleprices = saleprices + ['']
            stores = stores + ['California Carnivores']
            soldouts = soldouts + ['']
            species = species + [specie]
        
        #grabs information of sale items
        for container in containers_sale:
            name = container.find('p',attrs={'class':'grid-link__title'}).text.strip()
            specie = check_species(name)

            price = container.find('p',attrs={'class':'grid-link__meta'}).text.strip() #price of items on sale should be of form '$ xx.xx\n$ xx.xx'
            separate_index = price.rfind('$') #from end of string, look for 1st instance of $
            price_before = price[separate_index:] #gets original price
            price_before = price_before[2:] #remove the first 2 characters of the string '$ '
            price_after = price[:separate_index] #gets sale price
            price_after = price_after[2:] #remove the first 2 characters of the string '$ '
            price_after = price_after[:-1] #remove the last 2 characters of the string '$ '
    
            names = names + [name]
            prices = prices + [price_before]
            saleprices = saleprices + [price_after]
            stores = stores + ['California Carnivores']
            soldouts = soldouts + ['']
            species = species + [specie]
        
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            path = soup.find('a',attrs={'title':'Next »'})
            path = path['href']
            URL2 = URL + path
        except TypeError:
            page_next = False
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
# df1 = pd.DataFrame(d)
# df = df.append(df1,ignore_index=True)
pd.DataFrame(d)

,Name,Price,Sale Price,Sold Out,Species,Store
0,Carnivorous Plant Conservation Donation,1.00,,,other,California Carnivores
1,Dionaea m. 'Alien' Potted,49.99,,,dionaea muscipula,California Carnivores
2,Dionaea m. 'Bohemian Garnet' Potted,14.99,,,dionaea muscipula,California Carnivores
3,Dionaea m. 'Coquillage' Potted,24.99,,,dionaea muscipula,California Carnivores
4,Dionaea m. 'Funnel Trap' Potted,14.99,,,dionaea muscipula,California Carnivores
5,Dionaea m. 'Galaxy' Potted,39.99,,,dionaea muscipula,California Carnivores
6,Dionaea m. 'Korean Melody Shark' Potted,14.99,,,dionaea muscipula,California Carnivores
7,Dionaea m. 'Pink' Potted,14.99,,,dionaea muscipula,California Carnivores
8,Dionaea m. 'Red Piranha' Potted,14.99,,,dionaea muscipula,California Carnivores
9,Dionaea m. 'Tiger Jaws x Sawtooth' Potted,14.99,,,dionaea muscipula,California Carnivores


In [72]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Cook's Carnivorous Plants
URL = 'http://www.flytraps.com/Scripts/'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('span',attrs={'class':'CPcatDescProd'})
links = links[:22] #from cobra lily to nepenthes hybrid lowland

for link in links:
    #Checks what type of species
    kind = link.text
    specie = check_species(kind)
        
    #Create URL2 to be URL of species type
    path = link.a['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('div',attrs={'class':'prod-classic'})
    containers_count = len(containers)
    
    #print(URL2)
    #If is not blank, scrapes info. otherwise skips page
    if containers_count != 0:
        
        #each page contains 15 items, goes to next page when page_next mod 15 == 0
        page_next = 0
        while page_next % 15 == 0:
            page = urlopen(URL2)
            soup = BeautifulSoup(page,'html.parser')
            containers = soup.findAll('div',attrs={'class':'prod-classic'})
    
            for container in containers:
                name = container.h3.text.strip()
            
                #Most items on site are on sale. Try to process items as if the were on sale. If not on sale, then exception
                try:
                    price = container.find('del',attrs={'class':'CPprodLPriceV'}).text.strip()
                    price = price[1:] #removed '$' at beginning
            
                    saleprice = container.find('span',attrs={'class':'price'}).text.strip()
                    separate_index = saleprice.find('$') #look for 1st instance of $
                    saleprice = saleprice[separate_index+1:] #remove the all characters before and including the string '$'
                    saleprice = saleprice[:-3] #remove ' ea' from end of sales price
                except AttributeError:
                    price = container.find('span',attrs={'class':'price'}).text.strip()
                    separate_index = price.find('$') #look for 1st instance of $
                    price = price[separate_index+1:] #remove the all characters before and including the string '$'
                    price = price[:-3] #remove ' ea' from end of sales price
                
                    saleprice = ''
            
                names = names + [name]
                prices = prices + [price]
                stores = stores + ['Cook\'s Carnivorous Plants']
                soldouts = soldouts + ['']
                species = species + [specie]
                saleprices = saleprices + [saleprice]
                page_next = page_next + 1
                
            #Updates the URL for next page; stops the while loop if there is no next page
            if page_next % 15 == 0:
                paths = soup.find('div',attrs={'class':'spacing fl-right'})
                paths = paths.findAll('a')
                path = paths[len(paths)-1]['href'] #href of the last link (should be next page)
                URL2 = URL + path
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
# df1 = pd.DataFrame(d)
# df = df.append(df1,ignore_index=True)
pd.DataFrame(d)

,Name,Price,Sale Price,Sold Out,Species,Store
0,Darlingtonia Californica Cobra Lily unpotted s...,17.00,12.00,,darlingtonia,Cook's Carnivorous Plants
1,Darlingtonia Californica Cobra Lily unpotted n...,28.00,20.00,,darlingtonia,Cook's Carnivorous Plants
2,drosera scorpiodes unpotted near flowering siz...,5.00,3.25,,drosera,Cook's Carnivorous Plants
3,Cephalotus follicularis Charles Brewer potted ...,45.00,25.00,,cephalotus,Cook's Carnivorous Plants
4,Heliamphora purpurescens x ionasii small form ...,150.00,110.00,,heliamphora,Cook's Carnivorous Plants
5,venus flytrap bigmouth unpotted 2-3 year old s...,10.00,7.00,,dionaea muscipula,Cook's Carnivorous Plants
6,typical venus flytrap unpotted 2-3 year old si...,2.45,1.40,,dionaea muscipula,Cook's Carnivorous Plants
7,venus flytrap g14 unpotted 2-3 year old plant ...,14.00,12.00,,dionaea muscipula,Cook's Carnivorous Plants
8,typical venus flytrap unpotted near flowering ...,5.00,3.50,,dionaea muscipula,Cook's Carnivorous Plants
9,venus flytrap low giant unpotted 2-3 year old ...,17.00,10.00,,dionaea muscipula,Cook's Carnivorous Plants


In [75]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Carnivero
URL = 'https://www.carnivero.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('div',attrs={'class':'grid__item small--one-half medium-up--one-fifth slide-up-animation animated'})
links = [links[i] for i in range(10) if i!=1]  #all categories except beginner plants

#Looks at plant categories page
for link in links:            
    #Create URL2 to be URL of species type
    path = link.a['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
  
    page_next = True
    while page_next:
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('div',attrs={'class':'product grid__item medium-up--one-third small--one-half slide-up-animation animated'})
    
        for container in containers:
            name = container.find('div',attrs={'class':'product__title text-center'}).text.strip()
            specie = check_species(name)

            try:
                price = container.find('span',attrs={'class':'product__price'}).text.strip()
                separate_index = price.find('$') #look for 1st instance of $
                price = price[separate_index+1:] #remove the all characters before and including the string '$'
                prices = prices + [price]
                saleprices = saleprices + ['']           
            except AttributeError:
                #grab original price
                price = container.find('s').text.strip()
                separate_index = price.find('$') #look for 1st instance of $
                price = price[separate_index+1:] #remove the all characters before and including the string '$'
                prices = prices + [price]
            
                #grab new sales price
                price = container.find('span',attrs={'class':'product__price--on-sale'}).text.strip()
                separate_index = price.find('$') #look for 1st instance of $
                price = price[separate_index+1:] #remove the all characters before and including the string '$'
                saleprices = saleprices + [price] 
            
            try:
                soldout = container.find('strong',attrs={'class':'sold-out-text'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
            
            names = names + [name]
            stores = stores + ['Carnivero']
            species = species + [specie]
        
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            path = soup.find('span',attrs={'class':'next'})
            path = path.a['href']
            URL2 = 'https://www.carnivero.com' + path
        except AttributeError:
            page_next = False
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
# df1 = pd.DataFrame(d)
# df = df.append(df1,ignore_index=True)
pd.DataFrame(d)

,Name,Price,Sale Price,Sold Out,Species,Store
0,"Nepenthes veitchii Bareo, BE-3734",69.99,,sold out,nepenthes,Carnivero
1,"Nepenthes veitchii (""Big Mama"" x ""Pink Candy S...",59.99,49.99,,nepenthes,Carnivero
2,"Nepenthes maxima, BE-3067",34.99,24.99,,nepenthes,Carnivero
3,"Nepenthes maxima x ventricosa, CC",44.99,34.99,,nepenthes,Carnivero
4,"Nepenthes 'Splendid Diana' x ventricosa red, C...",44.99,34.99,sold out,nepenthes,Carnivero
5,"Nepenthes veitchii ""Pink""",74.99,,sold out,nepenthes,Carnivero
6,Nepenthes sibuyanensis variegated,400.00,,sold out,nepenthes,Carnivero
7,Nepenthes Trusmadiensis x burbidgeae - DM032,99.99,,sold out,nepenthes,Carnivero
8,Nepenthes (ventricosa x sibuyanensis) x macfar...,29.99,19.99,sold out,nepenthes,Carnivero
9,Nepenthes reinwardtiana,24.99,19.99,sold out,nepenthes,Carnivero


In [76]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Native Exotics
URL = 'https://nativeexoticsonline.com'
h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
req = Request(URL,headers=h) 
page = urlopen(req).read() 
soup = BeautifulSoup(page,'html.parser')
links = soup.find('ul',attrs={'class':'dropdown'})
links = links.findAll('a')
pages_of_interest = [5] + list(range(10,15)) #All Neps + Cephs,Drosera,..., Utrics
links = [links[i] for i in pages_of_interest]

#Looks at plant categories page
for link in links:      
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page_next = True
    while page_next:
        req = Request(URL2,headers=h) 
        page = urlopen(req).read() 
        soup = BeautifulSoup(page,'html.parser')
        container_big = soup.find('ul',attrs={'class':'products columns-4'})
        containers = container_big.findAll('li')
    
        for container in containers:
            name = container.find('h2').text.strip()
            specie = check_species(name)
        
            price_list = container.findAll('span',attrs={'class':'woocommerce-Price-amount amount'})
            #2 prices listed when on sale. 1st list element is original price, 2nd element is sales price
            if len(price_list) == 2:
                price = price_list[0].text.strip()
                price = price[2:]
                saleprice = price_list[1].text.strip()
                saleprice = saleprice[2:]             
            elif len(price_list) == 1:
                price = price_list[0].text.strip()
                price = price[2:]
                saleprice = ''
        
            try:
                soldout = container.find('span',attrs={'class':'soldout'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
            
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Native Exotics']
            saleprices = saleprices + [saleprice]
            species = species + [specie]
            
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            URL2 = soup.find('a',attrs={'class':'next page-numbers'})
            URL2 = URL2['href']
        except TypeError:
            page_next = False
        
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
# df1 = pd.DataFrame(d)
# df = df.append(df1,ignore_index=True)
pd.DataFrame(d)

,Name,Price,Sale Price,Sold Out,Species,Store
0,"Nepenthes maxima – dark x veitchii – Bario, sq...",99.00,89.10,,nepenthes,Native Exotics
1,Nepenthes spathulata x peltata – Seed Grown – ...,24.00,21.60,,nepenthes,Native Exotics
2,Nepenthes spectabilis – Seed Grown – BE-3768 –...,29.00,26.10,,nepenthes,Native Exotics
3,Nepenthes aenigma – BE-3770 – Borneo Exotics,24.00,21.60,,nepenthes,Native Exotics
4,Nepenthes maxima x aristolochioides – Borneo E...,19.00,17.10,,nepenthes,Native Exotics
5,Nepenthes ventricosa x mira – Christian Klein ...,26.00,23.40,,nepenthes,Native Exotics
6,Nepenthes aristolochioides x robcantleyi – BE-...,29.00,26.10,,nepenthes,Native Exotics
7,Nepenthes izumiae – Borneo Exotics – BE-3925 –...,49.00,44.10,,nepenthes,Native Exotics
8,Nepenthes jacquelineae – BE-3092 – Borneo Exot...,99.00,89.10,,nepenthes,Native Exotics
9,Nepenthes maxima x aristolochioides – Borneo E...,24.00,21.60,,nepenthes,Native Exotics


In [77]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Carnivorous Plant Nursery
URL = 'https://carnivorousplantnursery.com'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
links = soup.findAll('article')
links = links[:9] #Only looks at VFT to buterworts

#Looks at plant categories page
for link in links:      
    #Create URL2 to be URL of species type
    path = link.find('figure')
    path = path.a['href']
    URL2 = URL + path   
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page = urlopen(URL2)
    soup = BeautifulSoup(page,'html.parser')
    containers = soup.findAll('article',attrs={'class':'productgrid--item imagestyle--cropped-small '})
    for container in containers:
        name = container.find('h2').text.strip()
        specie = check_species(name)
    
        price = container.find('div',attrs={'class':'price--main'}).text.strip()
        separate_index = price.find('$') #look for 1st instance of $
        price = price[separate_index+1:] #remove the all characters before and including the string '$'
    
        names = names + [name]
        prices = prices + [price]
        stores = stores + ['Carnivorous Plant Nursery']
        saleprices = saleprices + [''] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!change when on sale
        species = species + [specie]
        
        #looks for sold out items
        try:
            soldout = container.find('span',attrs={'class':'productitem--badge badge--soldout'}).text.strip()
            soldouts = soldouts + [soldout.lower()]
        except AttributeError:
            soldouts = soldouts + ['']
            

d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
# df1 = pd.DataFrame(d)
# df = df.append(df1,ignore_index=True) 
pd.DataFrame(d)

,Name,Price,Sale Price,Sold Out,Species,Store
0,Venus Flytrap - Typical,5.95,,,dionaea muscipula,Carnivorous Plant Nursery
1,Venus Flytrap - King Henry,7.95,,,dionaea muscipula,Carnivorous Plant Nursery
2,Venus Flytrap - B52,11.95,,,dionaea muscipula,Carnivorous Plant Nursery
3,Venus Flytrap - Dentate Traps,5.95,,,dionaea muscipula,Carnivorous Plant Nursery
4,Venus Flytrap - Akai ryu,7.95,,,dionaea muscipula,Carnivorous Plant Nursery
5,Venus Flytrap - Big Vigorous,11.95,,,dionaea muscipula,Carnivorous Plant Nursery
6,Venus Flytrap - Red Dragon,9.95,,,dionaea muscipula,Carnivorous Plant Nursery
7,Venus Flytrap - Fast,9.95,,sold out,dionaea muscipula,Carnivorous Plant Nursery
8,Venus Flytrap - Pink Venus,11.95,,sold out,dionaea muscipula,Carnivorous Plant Nursery
9,Venus Flytrap - Czech Giant,13.95,,,dionaea muscipula,Carnivorous Plant Nursery


In [6]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Bergen Water Gardens
#cannot extract next page button functionality; cannot extract how many pages total
#instead keeps going to next page until error
try:
    page_number = 1
    while page_number < 20:
        URL = 'https://bergenwatergardens.com/product-category/carnivorous-plants/?fwp_paged=%d' % page_number
        h = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
        req = Request(URL,headers=h) 
        page = urlopen(req).read() 
        soup = BeautifulSoup(page,'html.parser')
    
        #wait random time between 0-5 seconds before scraping data
        r = random.randint(0,5)
        time.sleep(r)
        
        containers = soup.find('ul',attrs={'class':'products columns-4'})
        containers = containers.findAll('li')        
        
        for container in containers:
            name = container.find('h2').text.strip()
            specie = check_species(name)

            price = container.find('span',attrs={'class':'price'}).text.strip()
            price = price[2:] #removes '$ ' before price
    
            names = names + [name]
            prices = prices + [price]
            stores = stores + ['Bergen Water Gardens']
            saleprices = saleprices + [''] #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!change when on sale
            species = species + [specie]        
        
            #looks for sold out items
            try:
                URL2 = container.a['href']
                req = Request(URL2,headers=h) 
                page = urlopen(req).read() 
                soup = BeautifulSoup(page,'html.parser')
                
                soldout = soup.find('p',attrs={'class':'stock out-of-stock'}).text.strip()
                soldouts = soldouts + [soldout.lower()]
            except AttributeError:
                soldouts = soldouts + ['']
                        
        #increase page_number for while loop
        page_number = page_number + 1
except (AttributeError,IndexError):
    print('done') #when there is no page, while loop stops
    


d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
df = pd.DataFrame(d)
df

done


,Name,Price,Sale Price,Sold Out,Store,Species
0,72 Hour Heat Pack,3.00,,,Bergen Water Gardens,other
1,Akai Ryu Venus Fly Trap,12.00,,,Bergen Water Gardens,dionaea muscipula
2,Bugout™,30.00,,out of stock,Bergen Water Gardens,other
3,Cape Sundew Drosera capensis,12.00,,out of stock,Bergen Water Gardens,drosera
4,Fraser Island Sundew Drosera spatulata,8.00,,out of stock,Bergen Water Gardens,drosera
5,Nepenthes ‘Bill Bailey’ BE3820,15.00,,,Bergen Water Gardens,nepenthes
6,Nepenthes ‘Bill Bailey’ x robcantleyi BE3965,30.00,,,Bergen Water Gardens,nepenthes
7,Nepenthes (veitchii x lowii) x mira BE3910,35.00,,,Bergen Water Gardens,nepenthes
8,Nepenthes (veitchii x lowii) x spectabilis BE3400,25.00,,,Bergen Water Gardens,nepenthes
9,Nepenthes albomarginata BE3004,20.00,,out of stock,Bergen Water Gardens,nepenthes


In [ ]:
names = []
prices = []
saleprices = []
soldouts = []
stores = []
species = []

#Petflytrap
URL = 'https://www.petflytrap.com/All-carnivorous-plants_c_11.html'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
soup = soup.find('ul',attrs={'class':'sidebar-module__list'})
links = soup.findAll('a')
links = [links[i] for i in range(len(links)) if i not in (0,1,3,13,16,22,26,30,31,32,33,34,35,36,37)]
#exluding following pages: plant collections (0), easy to grow (1), drosera (3), sarracenia (13),
#nepenthes (16), pinguicula (22), utricularia (26), waterwheel, ..., gifts (30-37)

for link in links:
    #Create URL2 to be URL of species type
    path = link['href']
    URL2 = URL + path
    
    #wait random time between 0-5 seconds before scraping data
    r = random.randint(0,5)
    time.sleep(r)
    
    page_next = True
    while page_next:
        page = urlopen(URL2)
        soup = BeautifulSoup(page,'html.parser')
        containers = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half'})
        containers_sale = soup.findAll('div',attrs={'class':'grid__item large--one-quarter medium-down--one-half on-sale'})

        #grabs information of normal items
        for container in containers:
            name = container.text
            separate_index = name.rfind('Y') #finds the index for 'Y' to remove everything after 'Your Price...'
            separate_index
            name = name[:separate_index] #removes all text after 'Your Price'
            name = name[6:-4] #removes all newlines before and after name
            specie = check_species(name)

            price = container.find('td',attrs={'class':'price-info'}).text.strip()
            #if item is on sale, seperate into price_before and price_after
            if 'On sale' in price:
                separate_index = price.find('$') #from BEGINNING of string, look for 1st instance of $
                price_before = price[separate_index:] #gets original price
                price_before = price_before[1:6] #remove everything except price
    
                separate_index = price.rfind('$') #from END of string, look for 1st instance of $
                price_after = price[separate_index:] #gets original price
                price_after = price_after[1:6] #remove everything except price
            else:
                separate_index = price.rfind('$') #from end of string, look for 1st instance of $
                price_before = price[separate_index:] #gets original price
                price_before = price_before[1:] #remove the first 2 characters of the string '$ '
                price_after = ''
    
            names = names + [name]
            prices = prices + [price_before]
            saleprices = saleprices + [price_after]
            stores = stores + ['Petflytap']
            soldouts = soldouts + ['']
            species = species + [specie]
        
        #Updates the URL for next page; stops the while loop if there is no next page
        try:
            path = soup.find('a',attrs={'title':'Next »'})
            path = path['href']
            URL2 = URL + path
        except TypeError:
            page_next = False
        
d = {'Name':names,'Price':prices,'Sale Price':saleprices,'Sold Out':soldouts,'Store':stores,'Species':species}
# df1 = pd.DataFrame(d)
# df = df.append(df1,ignore_index=True)
pd.DataFrame(d)

In [22]:
URL = 'https://www.petflytrap.com/All-carnivorous-plants_c_11.html'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('td',attrs={'width':'20%'})
container = containers[0]
container

<td width="20%">
<table border="0" cellpadding="5" cellspacing="0" class="alternative" width="99%">
<tr>
<td align="center" height="150"><a href="Drosera-binata--forked-sundew_p_101.html"><img alt="Drosera binata - forked sundew" border="0" src="thumbnail.asp?file=assets/images/drbinasp_thumbnail.jpg&amp;maxx=0&amp;maxy=150"/></a></td>
</tr>
<tr>
<td align="center" class="item" height="60"><a href="Drosera-binata--forked-sundew_p_101.html">Drosera binata - forked sundew</a></td>
</tr>
<tr>
<td align="center" class="price-info" valign="top"><!--START: ITEMPRICE-->
<b>Your Price: </b><span class="price">$6.99</span>
<!--END: ITEMPRICE-->
</td>
</tr>
<tr>
<td align="center" class="item" valign="middle"><input class="btn" onclick="window.location='add_cart.asp?quick=1&amp;item_id=101'" onmouseout="this.className='btn'" onmouseover="this.className='btn_over'" type="button" value="Add To Cart"/></td>
</tr>
</table>
</td>

In [84]:
container = containers[7]
name = container.text
separate_index = name.rfind('Y') #finds the index for 'Y' to remove everything after 'Your Price...'
separate_index
name = name[:separate_index] #removes all text after 'Your Price'
name = name[6:-4] #removes all newlines before and after name
name


price = container.find('td',attrs={'class':'price-info'}).text.strip()
#if item is on sale, seperate into price_before and price_after
if 'On sale' in price:
    separate_index = price.find('$') #from BEGINNING of string, look for 1st instance of $
    price_before = price[separate_index:] #gets original price
    price_before = price_before[1:6] #remove everything except price
    
    separate_index = price.rfind('$') #from END of string, look for 1st instance of $
    price_after = price[separate_index:] #gets original price
    price_after = price_after[1:6] #remove everything except price
else:
    separate_index = price.rfind('$') #from end of string, look for 1st instance of $
    price_before = price[separate_index:] #gets original price
    price_before = price_before[1:] #remove the first 2 characters of the string '$ '
    price_after = ''

price_after

'32.99'

In [95]:
path = soup.findAll('font',attrs={'size':'1'})
path[len(path)-1].a['href']
#soup.findAll('font',attrs={'width':'20%'})

'All-carnivorous-plants_c_11-2-3.html'

In [ ]:
#Once the complete database is compiled, apply tagging function - tags any species found in product
#https://www.carnivorousplants.co.uk/resources/nepenthes-interactive-guide/
#https://en.wikipedia.org/wiki/List_of_Nepenthes_species

#compare product to database of species known
#if str in database, then species = str

In [ ]:
n = 172
name_nepenthes = [''] * n
URL = 'https://en.wikipedia.org/wiki/List_of_Nepenthes_species'
page = urlopen(URL)
soup = BeautifulSoup(page,'html.parser')
containers = soup.findAll('tr')

for i in range(1,n):
    container = containers[i]
    name = container.find('a')
    name_nepenthes[i] = name['title'][10:] #removing 'Nepenthes ' from string
    
name_nepenthes = name_nepenthes[1:] #remove header row

In [ ]:
#same as cell above, but for venus fly traps
#wip
r = random.randint(0,5)
print(r)
time.sleep(r)
1+1

In [ ]:
df['Cross1'] = ''
df['Cross2'] = ''
df['Cross3'] = ''
df['Cross4'] = ''
df['Cross5'] = ''
df['Cross6'] = ''

#i is row index, j is column index
#len(df) gives number of rows in df
for i in range(len(df)):
    string_split = df.at[i,'Name'].replace('(','').replace(')','').replace('[','').replace(']','').replace(',','')
    #removing all of the special characters:()[] so the string is split into 'ventricosa' and not '(ventricosa''
    string_split = string_split.split(' ')
    count = 1                           #counter for column name
    for j in range(len(string_split)):
        if string_split[j] in name_nepenthes:
            column = 'Cross%d' % count
            count = count + 1
            df.loc[[i],[column]] = string_split[j]

            
            

#also consider the cases for natural/common hybrids: ventrata, miranda
#create special if clause when tag is not found, open product page and scan description text for tags
df

In [ ]:
#Check what does not have any Cross1 tag
for i in range(len(df)):
    if df.at[i,'Cross1'] is '':
        print(df.at[i,'Name'])